# Ejemplo CVA

Se calcula el CVA de un swap.

## Librerías

- Las funciones relacionadas con el modelo de Hull-White que definimos y utilizamos en los notebooks 12 y 13 han sido mejoradas y encapsuladas en el módulo `hull_white.py`.

- La función `bono_tasa_fija` se trasladó al nuevo módulo `instruments.py`.

In [1]:
from modules import instruments as inst
from modules import hull_white as hw
from modules import auxiliary as aux
import pandas as pd
import numpy as np
import math

## Auxiliares

In [2]:
frmt = {
    'plazo': '{:,.0f}',
    'tasa': '{:.8%}',
    'df': '{:.6%}',
    't': '{:.4f}'
}

## Data Curva

Se lee la data y se completa con el plazo en convención Act/365.

In [3]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [4]:
df_curva['t'] = df_curva['plazo'] / 365.0

In [6]:
df_curva.head().style.format(frmt)

,plazo,tasa,df,t
0,1,0.08111102%,99.999778%,0.0027
1,7,0.08405071%,99.998388%,0.0192
2,14,0.07796689%,99.997010%,0.0384
3,21,0.07735805%,99.995549%,0.0575
4,33,0.07806669%,99.992942%,0.0904


## Funciones para el Modelo de HW

- `zrate`: es el cubic spline construido a partir de las columnas `t` y `tasa` de `df_curva`.
- `hwz`: es una versión más sencilla de la función `zero_hw` que sólo requiere llos parámetros `r`, `t` y `T`.
- `theta`: es la misma función que ya utilizamos.

In [7]:
gamma = 1.0
sigma = .0025

Se construyen las funciones `zrate`, `hwz` y `theta`.

In [8]:
zrate, hwz, theta = hw.get_zrate_hwzero_and_theta(
    df_curva['t'],
    df_curva['tasa'],
    gamma,
    sigma
)

In [12]:
def demo():
    return 1, 2

In [19]:
uno, dos = demo()

In [20]:
tupla = demo()

In [21]:
tupla

(1, 2)

Se verifica que `zrate` y más importante aún, `hwz` coincidan con los valores de la curva de mercado en los vértices de interpolación (plazos y tasas de la curva).

**Notar** el plazo utilizado para obtener el valor de `r0`.

In [22]:
r0 = zrate(.0000001)
print(f'         r0: {r0: .6%}\n')
for tup in df_curva.itertuples():
    print(f'      plazo: {tup.t:.4f}')
    print(f' tasa curva: {tup.tasa:.6%}')
    print(f'tasa spline: {zrate(tup.t):.6%}')
    print(f'   tasa hwz: {-math.log(hwz(r0, 0, tup.t)) / tup.t:.6%}')
    print()

         r0:  0.078814%

      plazo: 0.0027
 tasa curva: 0.081111%
tasa spline: 0.081111%
   tasa hwz: 0.081111%

      plazo: 0.0192
 tasa curva: 0.084051%
tasa spline: 0.084051%
   tasa hwz: 0.084051%

      plazo: 0.0384
 tasa curva: 0.077967%
tasa spline: 0.077967%
   tasa hwz: 0.077967%

      plazo: 0.0575
 tasa curva: 0.077358%
tasa spline: 0.077358%
   tasa hwz: 0.077358%

      plazo: 0.0904
 tasa curva: 0.078067%
tasa spline: 0.078067%
   tasa hwz: 0.078067%

      plazo: 0.1671
 tasa curva: 0.078064%
tasa spline: 0.078064%
   tasa hwz: 0.078064%

      plazo: 0.2521
 tasa curva: 0.081103%
tasa spline: 0.081103%
   tasa hwz: 0.081103%

      plazo: 0.3425
 tasa curva: 0.078059%
tasa spline: 0.078059%
   tasa hwz: 0.078059%

      plazo: 0.4164
 tasa curva: 0.076030%
tasa spline: 0.076030%
   tasa hwz: 0.076030%

      plazo: 0.4986
 tasa curva: 0.075014%
tasa spline: 0.075014%
   tasa hwz: 0.075014%

      plazo: 0.5808
 tasa curva: 0.073593%
tasa spline: 0.073593%
   tasa h

Se verifica que la función `theta` se mantenga acotada.

In [23]:
for tup in df_curva.itertuples():
    print(f'theta({tup.t:.4f}): {theta(tup.t):.6%}')

theta(0.0027): 1.287414%
theta(0.0192): -0.943991%
theta(0.0384): 0.800401%
theta(0.0575): 0.047061%
theta(0.0904): 0.019791%
theta(0.1671): 0.337408%
theta(0.2521): -0.310565%
theta(0.3425): 0.153700%
theta(0.4164): 0.125659%
theta(0.4986): -0.041470%
theta(0.5808): 0.079302%
theta(0.6658): 0.125025%
theta(0.7479): 0.024419%
theta(0.8384): 0.071200%
theta(0.9178): 0.032841%
theta(1.0000): 0.030964%
theta(1.4986): -0.021227%
theta(2.0000): 0.147546%
theta(3.0055): 0.359198%
theta(4.0055): 0.542529%
theta(5.0027): 0.840753%
theta(6.0027): 0.932951%
theta(7.0027): 1.040001%
theta(8.0110): 1.159568%
theta(9.0082): 1.286390%
theta(10.0082): 1.251110%
theta(12.0082): 1.260508%
theta(15.0110): 1.306324%
theta(20.0164): 1.164882%
theta(25.0219): 1.118646%
theta(30.0192): 0.996521%
theta(40.0274): 0.535297%
theta(50.0329): 0.495002%


## Simulación

Se definen los parámetros requeridos, tanto del modelo como de la simulación.

In [24]:
num_sims = 1000
num_steps = 528 # 264 * 2 (2 años)

Se obtienen las trayectorias.

In [25]:
time_steps, paths = hw.sim_hw_many(gamma, sigma, theta, r0, 1000, 528, seed=1000)

**Ejercicio:** construir una función que retorne la función `sim_hw_many` pero sólo con los argumentos r0, num_sims, num_:steps y seed. O sea, que gamma, sigma y theta queden encerrados (closure) dentro de la función.

In [27]:
time_steps[264:284]

array([1.        , 1.00378788, 1.00757576, 1.01136364, 1.01515152,
       1.01893939, 1.02272727, 1.02651515, 1.03030303, 1.03409091,
       1.03787879, 1.04166667, 1.04545455, 1.04924242, 1.0530303 ,
       1.05681818, 1.06060606, 1.06439394, 1.06818182, 1.0719697 ])

In [28]:
paths[0][0:10] # son los primeros 10 valores de la primera trayectoria simulada.

array([0.00078814, 0.00073679, 0.0008229 , 0.00082631, 0.00091057,
       0.00083378, 0.00085509, 0.0007984 , 0.00069011, 0.00076015])

Se extrae el elemento correspondiente a 6M, 1Y, 18M de cada simulación:

EE = Promedio(max(Vi, 0)) y se va a calcular en estos tiempos de parada o stopping times.

In [37]:
last_r = [(s[132], s[264], s[396]) for s in paths]

In [57]:
last_r

[(0.00023007861920438662, -0.00228613411685056, -0.0010280821758350091),
 (0.0025169292332102976, -0.0011854362051575008, -0.0001687879785847798),
 (0.0027664535202960713, -0.00014629890902674244, -0.0014560621172352867),
 (0.001319077993090508, 0.0002903365861429921, 0.0003031121774537),
 (0.0013310969798680174, 0.0016605224151117776, 0.00046149375927135466),
 (-0.00047824174832072604, -7.491262281426666e-05, -0.002372965361821875),
 (-0.0006990499021550815, 0.0008701005065045559, 0.0023820764482183673),
 (-0.00024242315645648245, -0.0005241506091345296, -0.0010662074667958767),
 (0.002971131308690492, 0.0012623918635773213, 0.001006548774328804),
 (0.0017895576578377578, 0.001858793539411228, 0.0038551492363327567),
 (-0.0003874349085115471, -0.0017529925452902495, -0.00027573503626153613),
 (0.0006400359350959788, -0.0007307890472739373, 0.0011590567354975853),
 (-0.0009335783757506635, -0.0008496383555476353, -0.001659369075500176),
 (0.0024510592164944363, 0.0044413586668778085, 0

## Cálculo de la *EE* de un Swap

Para el swap vamos a utilizar una función sencilla para la pata fija y consideraremos que el valor presente de la pata flotante siempre es igual al nocional. Consideremos un swap a 2Y.

In [40]:
swap_fija = inst.bono_tasa_fija(0, .5, 4, .001)

In [41]:
swap_fija

[(0.5, 0.05), (1.0, 0.05), (1.5, 0.05), (2.0, 100.05)]

Se define una función que construye una función que valoriza el swap a partir de $t$ y $r_t$.

In [43]:
def valorizador_swap(pata_fija, df_function):
    def valor(r, t):
        result = 0.0
        for e in pata_fija:
            if e[0] > t:
                # Si la condición fuera e[0] >= t, lo que significa
                # que se incluiría el valor de un cupón que se paga
                # en t, entonces en la línea (alpha) tendría que incluir
                # el valor del cupón de la pata flotante. La fórmula sería,
                # result - (100 + cupon_pata_flotante(t))
                result += e[1] * df_function(r, t, e[0])
        return result - 100.0 if result > 0.0 else 0.0 # (alpha)
    return valor

Probemos:

In [44]:
vswap = valorizador_swap(swap_fija, hwz)

In [51]:
vswap(r0, 1)

0.04608515685097814

Valoricemos el swap en 6M más en cada simulación:

In [52]:
valor_swap_6m = [vswap(r[0], .5) for r in last_r] # List comprehensions: es técnica de programación
                                                  # funcional.

In [58]:
temp = []
for r in last_r:
    # Porqué estoy haciendo esto ...
    temp.append(vswap(r[0], .5))

In [59]:
temp[0:10]

[0.1087600975840104,
 -0.06887346705883601,
 -0.08823644782663109,
 0.024131556838455026,
 0.023197934213627036,
 0.16384366805613126,
 0.18102133440184787,
 0.14550152472715183,
 -0.10411655758359473,
 -0.012408317120417678]

In [54]:
valor_swap_6m[0:10]

[0.1087600975840104,
 -0.06887346705883601,
 -0.08823644782663109,
 0.024131556838455026,
 0.023197934213627036,
 0.16384366805613126,
 0.18102133440184787,
 0.14550152472715183,
 -0.10411655758359473,
 -0.012408317120417678]

Mejor aún, podemos calcular directamente la exposición.

In [55]:
e_swap_6m = [max(vswap(r[0], .5), 0) for r in last_r]

In [56]:
e_swap_6m[0: 10]

[0.1087600975840104,
 0,
 0,
 0.024131556838455026,
 0.023197934213627036,
 0.16384366805613126,
 0.18102133440184787,
 0.14550152472715183,
 0,
 0]

Por lo tanto, la exposición esperada en 6M es:

In [60]:
ee_swap_6m = np.average(e_swap_6m)

In [61]:
print(f'La EE del swap en 6M es: {ee_swap_6m: ,.2f}')

La EE del swap en 6M es:  0.09


**Ejercicio:** completar el cálculo del CVA. Asumir RR y PDs arbitrarias y eventualmente sensibilizar el resultado a estos parámetros.

Se calcula la *EE* a 1Y y a 18M.

In [62]:
e_swap_1y = [max(vswap(r[1], 1), 0) for r in last_r]

In [63]:
e_swap_1y[0: 10]

[0.24065829912002812,
 0.17095087350774918,
 0.10518658183410423,
 0.0775659442178096,
 0,
 0.10067031514724079,
 0.04090308337163151,
 0.12909483404341415,
 0.016103151244834635,
 0]

In [64]:
e_swap_18m = [max(vswap(r[2], 1.5), 0) for r in last_r]

In [65]:
e_swap_18m[0: 10]

[0.08860774044821085,
 0.05477290975665028,
 0.10546377945770757,
 0.03619663996914824,
 0.02996274884982597,
 0.1415856749666915,
 0,
 0.09010919423472785,
 0.008512379477650711,
 0]

Luego, se calcula el factor de descuento a cada uno de los plazos en que se calcula *EE*: 6M, 1Y y 18M.

- Se extraen los resultados de la simulación hasta 6M, 1Y y 18M.

In [71]:
paths_up_to_6m = [s[0:131] for s in paths] # Hasta 1 día antes del stopping time
paths_up_to_1y = [s[0:263] for s in paths]
paths_up_to_18m = [s[0:395] for s in paths]

In [72]:
paths_up_to_6m[0]

array([0.00078814, 0.00073679, 0.0008229 , 0.00082631, 0.00091057,
       0.00083378, 0.00085509, 0.0007984 , 0.00069011, 0.00076015,
       0.00068639, 0.00081364, 0.00071872, 0.0005628 , 0.00052394,
       0.00051075, 0.00052891, 0.00050694, 0.00061448, 0.00080838,
       0.0006538 , 0.00059999, 0.00058259, 0.00064331, 0.00078281,
       0.00082863, 0.0010622 , 0.00097446, 0.00091292, 0.00078417,
       0.00102902, 0.00101641, 0.00102771, 0.00097632, 0.00096888,
       0.00092224, 0.00121913, 0.00120758, 0.00111935, 0.00087286,
       0.00101055, 0.00085509, 0.00096136, 0.00138355, 0.00151324,
       0.00135141, 0.00133144, 0.00127353, 0.00122966, 0.00106533,
       0.00095785, 0.00136048, 0.00152982, 0.00124547, 0.00114887,
       0.00131779, 0.0013457 , 0.0012167 , 0.00095473, 0.00112768,
       0.00128558, 0.00095266, 0.00072585, 0.00074682, 0.00095324,
       0.0009874 , 0.00096219, 0.00104691, 0.00108171, 0.00119798,
       0.00103792, 0.00105058, 0.0008168 , 0.00074668, 0.00049

Luego, se calculan los factores de descuento "devolviéndose por cada trayectoria".

In [73]:
dt = 1 / 264.0   PROD(i=0, 131) exp(-ri(t) * dt)
                 exp(dt*SUMA(i=0, 131)ri(t))

In [74]:
dfs_up_to_6m = [np.exp(-dt * np.sum(path)) for path in paths_up_to_6m]
dfs_up_to_1y = [np.exp(-dt * np.sum(path)) for path in paths_up_to_1y]
dfs_up_to_18m = [np.exp(-dt * np.sum(path)) for path in paths_up_to_18m]

In [75]:
pv_ee_6m = [both[0] * both[1] for both in zip(e_swap_6m, dfs_up_to_6m)]
pv_ee_1y = [both[0] * both[1] for both in zip(e_swap_1y, dfs_up_to_1y)]
pv_ee_18m = [both[0] * both[1] for both in zip(e_swap_18m, dfs_up_to_18m)]

In [76]:
mean_pv_ee_6m = np.mean(pv_ee_6m)
mean_pv_ee_1y = np.mean(pv_ee_1y)
mean_pv_ee_18m = np.mean(pv_ee_18m)
print(mean_pv_ee_6m, mean_pv_ee_1y, mean_pv_ee_18m)

0.09064515573488405 0.07915491052290452 0.049815964719884996


**Pregunta estilo examen de grado**: describa con precisión este procedimiento e indique qué cambiaría si usara las oportunas medidas forward. 